<a href="https://colab.research.google.com/github/RodolfoFerro/ml-facilito/blob/main/notebooks/Deep_Learning_Clase_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning 101 - Clase 3  🧠

> **Descripción:** Cuaderno de contenidos (III) sobre introducción a _deep learning_ para el Bootcamp de Machine Learning con Código Facilito, 2023. <br>
> **Autor:** [Rodolfo Ferro](https://github.com/RodolfoFerro) <br>
> **Contacto:** [X](https://twitter.com/rodo_ferro) / [Instagram](https://www.instagram.com/rodo_ferro/)


## Contenido

### Sección VII

1. Keras Core
2. Tensores, operadores, funciones en keras_core.ops
3. ¿Por qué utilizar Keras Core?


### Sección VIII

4. Introducción a PyTorch
5. Tensores, operadores, funciones
6. El problema de separabilidad lineal - XOR
7. Redes neuronales con PyTorch


### Sección IX – Tarea

8. El dataset a utilizar
9. Preparación de los datos
10. Creación del modelo
11. Entrenamiento del modelo
12. Evaluación y predicción

## **Sección VII**

### **Keras Core**

[Keras](https://keras.io/) es una API diseñada para seres humanos, no para máquinas. Sigue las mejores prácticas para reducir la carga cognitiva: ofrece API simples y consistentes, minimiza la cantidad de acciones del usuario requeridas para casos de uso comunes y proporciona mensajes de error claros y procesables.

#### **¿Qué es Keras Core?**

- Una reescritura completa de Keras
  - Sin deuda técnica
  - Un codebase más pequeño (43K LOC, ~3x más pequeño)
- Con soporte multi-backend (JAX, TensorFlow, PyTorch, & Numpy)
  - El backend de Numpy es para inferencia solamente
- Reemplazo directo para tf.keras cuando usas TensorFlow como backend
- Funciona perfectamente con KerasNLP y KerasCV


In [ ]:
!pip install keras_core -q

In [ ]:
import numpy as np


x = np.array([(0, 0), (1, 0), (0, 1), (1, 1)])
y = np.array([0, 1, 1, 0])

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch" # or "tensorflow" or "jax"

import keras_core as keras
keras.ops.numpy.arange(5)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(2, activation='tanh', input_shape=(2, )),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
loss = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.SGD(learning_rate=0.6)
model.compile(optimizer=optimizer, loss=loss, metrics=[loss])

In [ ]:
history = model.fit(x, y, epochs=1000)

In [ ]:
import plotly.express as px


losses = history.history['loss']
eje_x = np.arange(len(losses))

fig = px.line(
    x=eje_x,
    y=losses,
    title='Historia de entrenamiento',
    labels=dict(x='Épocas', y='Error')
)
fig.show()

In [ ]:
# Construcción de una rejilla
x = np.linspace(-0.1, 1.1, 201)
y = np.linspace(-0.1, 1.1, 201)
xy = np.meshgrid(x, y)
zz = np.array(list(zip(*(x.flat for x in xy))))

# Predicción en la rejilla de valores
surface = model.predict(zz)
surface = surface.flatten()

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Scatter3d(
    x=zz[:, 0],
    y=zz[:, 1],
    z=surface,
    mode='markers',
    marker=dict(
        size=1,
        color=surface,
        colorscale='Viridis',
        opacity=0.8
    )
)])

# Tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

## **Sección VIII**

### **Introducción a PyTorch**

[PyTorch](https://pytorch.org/get-started/locally/) es una biblioteca de aprendizaje profundo open-source basada en la biblioteca de Torch (Lua), utilizado para aplicaciones como visión artificial y procesamiento de lenguajes naturales, principalmente desarrollado por el Laboratorio de Investigación de IA de Facebook.


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


# TODO. Load samples
x = np.array([(0, 0), (1, 0), (0, 1), (1, 1)])
y = np.array([[0], [1], [1], [0]])

tensor_x = None # TODO. Load torch.Tensor() to device
tensor_y = None # TODO. Load torch.Tensor() to device

dataset = TensorDataset(tensor_x, tensor_y)
train_dataloader = DataLoader(dataset)

### **La red neurona, el optimizador y la función de pérdida**

$$ \mathrm{MSE}=\frac{1}{N}\cdot\sum_{i=1}^N \left(y_i- \hat{y}_i \right )^2 $$

In [ ]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        # TODO. Create Sequential model
        self.model = nn.Sequential(
            # TODO. Linear() that maps 2 -> 2
            # TODO. Activation function
            # TODO. Linear() that maps 2 -> 1
            # TODO. Activation function
        )

    def forward(self, x):
        out = self.model(x.view(x.size(0), 2))
        out = out.view(out.size(0), -1)
        return out

In [ ]:
def train_net(model, loss_fn, optimizer, dataloader):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        # Compute prediction and loss
        # TODO. Prediction is computed with the model
        # TODO. Use loss_fn or use your own loss function
        # e.g. loss = (y - pred) ** 2 or loss_fn(y, pred)
        # loss.sum()

        # Backpropagation
        # TODO. optimizer -> zero_grad()
        # TODO. loss -> backward()
        # TODO. optimizer -> step()

    return loss.item()

- ¿Por qué hacer **zero_grad()**? https://stackoverflow.com/a/67819799

### **Entrenamiento de la red**

In [ ]:
model = NeuralNet().to(device)
print(model)

In [ ]:
from tqdm import tqdm


# Set learning rate, create optimizer and set loss fn
learning_rate = 0.2
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Set epochs and train model
epochs = 1000
losses = []
pbar = tqdm(range(epochs))
for t in pbar:
    loss = train_net(model, loss_fn, optimizer, train_dataloader)
    pbar.set_postfix({'Loss': loss})
    losses.append(loss)

In [ ]:
import plotly.express as px


eje_x = np.arange(len(losses))

fig = px.line(
    x=eje_x,
    y=losses,
    title='Historia de entrenamiento',
    labels=dict(x='Épocas', y='Error')
)
fig.show()

In [ ]:
# Construcción de una rejilla
x = np.linspace(-0.1, 1.1, 201)
y = np.linspace(-0.1, 1.1, 201)
xy = np.meshgrid(x, y)
zz = np.array(list(zip(*(x.flat for x in xy))))

# Predicción en la rejilla de valores
z = torch.Tensor(zz).to(device)
surface = model(z)

surface = surface.cpu().data.numpy().flatten()

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Scatter3d(
    x=zz[:, 0],
    y=zz[:, 1],
    z=surface,
    mode='markers',
    marker=dict(
        size=1,
        color=surface,
        colorscale='Viridis',
        opacity=0.8
    )
)])

# Tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

<center>
    *********
</center>

## **Sección III – Tarea**

### El dataset a utilizar: Naranjas vs. Manzanas

El dataset ha sido una adaptación de datos encontrados en [Kaggle](https://www.kaggle.com/datasets/theblackmamba31/apple-orange). Dicho dataset está compuesto por conjuntos de imágenes de naranjas y manzanas que serán un utilizados para entrenar una neurona artificial.


Para cargar los datos, primero los descargaremos de un repositorio donde previamente los preparé para ustedes.

Puedes explorar directamente los archivos fuente del [repositorio en GitHub – `apple-orange-dataset`](https://github.com/RodolfoFerro/apple-orange-dataset).

Puedes también explorar el [script](https://github.com/RodolfoFerro/apple-orange-dataset/blob/main/script.py) que he utilizado para la preparación de los mismos.

In [ ]:
!wget https://raw.githubusercontent.com/RodolfoFerro/apple-orange-dataset/main/training_data.csv
!wget https://raw.githubusercontent.com/RodolfoFerro/apple-orange-dataset/main/testing_data.csv

### Preparación de los datos


In [ ]:
import pandas as pd


training_df = pd.read_csv('training_data.csv')
testing_df = pd.read_csv('testing_data.csv')

training_df

In [ ]:
training_df['class_str'] = training_df['class'].astype('str')
training_df['hover'] = [text.split('/')[-1] for text in training_df['filename']]

testing_df['class_str'] = testing_df['class'].astype('str')
testing_df['hover'] = [text.split('/')[-1] for text in testing_df['filename']]

training_df

### Exploración de los datos

Podemos verificar si el conjunto de datos está balanceado:

In [ ]:
training_df.groupby('class').count()

Podemos explorar cómo se ven los datos en un gráfico 3D:

In [ ]:
import plotly.express as px


fig = px.scatter_3d(
    training_df,
    x='r', y='g', z='b',
    color='class_str',
    symbol='class_str',
    color_discrete_sequence=['#be0900', '#ffb447'],
    opacity=0.5,
    hover_data=['hover']
)
fig.show()

Puedes explorar las imágenes y sus valores de color utilizando el color picker que ofrece Google: https://g.co/kgs/uarXyu

> **Pregunta clave:** ¿Los datos son linealmente separables? Con lo que hemos explorado hasta ahora, ¿basta una neurona para resolver el problema planteado?

### Creación de una red neuronal artificial


In [ ]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            # TODO. Linear() that maps 3 -> 5
            # TODO. Activation function
            # TODO. Linear() that maps 5 -> 2
            # TODO. Activation function
            # TODO. Linear() that maps 2 -> 1
            # TODO. Activation function

        )

    def forward(self, x):
        out = self.model(x.view(x.size(0), 3))
        out = out.view(out.size(0), -1)
        return out

Antes de entrenar los datos, procedemos a escalarlos a valores en [0, 1].

In [ ]:
x = training_df[['r', 'g', 'b']].values / 255.
y = training_df['class'].values
y = y.reshape((len(y), 1))

tensor_x = torch.Tensor(x).to(device)
tensor_y = torch.Tensor(y).to(device)

batch_size = 64
dataset = TensorDataset(tensor_x, tensor_y)
train_dataloader = DataLoader(dataset, batch_size=batch_size)

In [ ]:
def train_net(model, loss_fn, optimizer, dataloader):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(x)
        # loss = - y * torch.log(pred) - (1 - y) * torch.log(1 - pred)
        # loss = loss.sum()
        loss = loss_fn(y, pred)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return loss.item()

In [ ]:
model = NeuralNet().to(device)


learning_rate = 0.0002


loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 1000
losses = []
pbar = tqdm(range(epochs))
for t in pbar:
    loss = train_net(model, loss_fn, optimizer, train_dataloader)
    pbar.set_postfix({'Loss': loss})
    losses.append(loss)

In [ ]:
import plotly.express as px


eje_x = np.arange(len(losses))

fig = px.line(
    x=eje_x,
    y=losses,
    title='Historia de entrenamiento',
    labels=dict(x='Épocas', y='Error')
)
fig.show()

### Predicciones con el modelo



> **Pregunta clave:** ¿Qué sucede con la historia de entrenamiento?

> **Pro-tip:** Exploremos con una nueva función de pérdida, qué tal la utilizada usualemente en una regresión logística: https://developers.google.com/machine-learning/crash-course/logistic-regression/model-training

Para predecir un color de ejemplo:

In [ ]:
# Preparamos los datos
sample_index = 1

input_sample = testing_df[['r', 'g', 'b']].iloc[sample_index].values
# input_sample = np.array([])
print('Color real:', input_sample)

input_sample = input_sample / 255.
input_sample = input_sample.reshape((1, len(input_sample)))
print('Color transformado:', input_sample)

real_class = testing_df[['class']].iloc[sample_index].values
print('Clase real:', real_class)

In [ ]:
input_sample = torch.Tensor(input_sample).to(device)

model(input_sample).cpu().data.numpy()[0, 0]

Para evaluar esta tarea, vamos a utilizar funciones de scikit-learn para la que nos permitirán realizar la evaluación de resultados en el conjunto de pruebas. (Utilizar [`sklearn.metrics.accuracy_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score))

<center>
    *********
</center>

In [ ]:
import plotly.express as px


fig = px.scatter_3d(
    testing_df,
    x='r', y='g', z='b',
    color='class_str',
    symbol='class_str',
    color_discrete_sequence=['#be0900', '#ffb447'],
    opacity=0.5,
    hover_data=['hover']
)
fig.show()

In [ ]:
def get_predictions(testing_df, threshold=0.5):
    testing_inputs = testing_df[['r', 'g', 'b']].values / 255.
    testing_output = testing_df['class'].values

    predictions = []
    for test_input in testing_inputs:
        test_input = test_input.reshape((1, len(test_input)))
        test_input = torch.Tensor(test_input).to(device)

        if model(test_input).cpu().data.numpy()[0, 0] <= threshold:
            prediction = 0
        else:
            prediction = 1
        predictions.append(prediction)
    predictions = np.array(predictions)

    return testing_output, predictions

In [ ]:
from sklearn.metrics import accuracy_score


testing_output, predictions = get_predictions(testing_df, threshold=0.8)
result = accuracy_score(testing_output, predictions)
print(f'Accuracy: {result * 100:.6}%')

> **Pregunta clave:** ¿Qué sucede si cambiamos el _threshold_ a 0.7? A veces conviene explorar el valor de umbral que seleccionamos y no siempre dar por hecho que 0.5 va a funcionar todas las veces. <br><br>
> Lee más aquí: https://ploomber.io/blog/threshold/

> **Para resolver la tarea, el reto es:** Mejor accuracy obtenido en la clase.

**Puedes explorar:**
- Utilizar 1 a 3 variables (de las dadas).
- Investigar e implementar una nueva función para estimar el error.
- Realizar transformaciones en los datos.
- Entrenar por más épocas.
- Mover el umbral para definir la clase.
- Explorar otras funciones de activación.
- Generar tu nuevo dataset de datos a partir de las imágenes originales.

--------

> Contenido creado por **Rodolfo Ferro**, 2023. <br>
> Puedes contactarme a través de Insta ([@rodo_ferro](https://www.instagram.com/rodo_ferro/)) o X ([@rodo_ferro](https://twitter.com/rodo_ferro)).